In [14]:
#! lamin init --storage ./lamin-intro --schema bionty 

import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scdataloader.utils import load_genes
import numpy as np
import anndata as ad
from huggingface_hub import hf_hub_download

from scprint.tasks import Denoiser, withknn
from scprint.model.utils import downsample_profile
from scipy.stats import spearmanr
from anndata import AnnData

import seaborn as sns
%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#model_checkpoint_file = hf_hub_download(
#    repo_id="jkobject/scPRINT", filename=f"v2-medium.ckpt"
#)
#model_checkpoint_file = ../data/
model_checkpoint_file = '../../../80vhz9uh_init.ckpt'

In [25]:
m = torch.load(model_checkpoint_file)
m['hyper_parameters']['label_decoders'] = {k:{u: str(w) for u,w in v.items()} for k,v in m['hyper_parameters']['label_decoders'].items()}
m['hyper_parameters']['organisms'] = ["NCBITaxon:10090", "NCBITaxon:9606"]
torch.save(m, model_checkpoint_file)

In [21]:
model = scPrint.load_from_checkpoint(model_checkpoint_file, use_metacell_token=True, precpt_gene_emb=None)

FYI: scPrint is not attached to a `Trainer`.


In [ ]:
m = torch.load(model_checkpoint_file)
if "prenorm" in m['hyper_parameters']:
    m['hyper_parameters'].pop("prenorm")
    torch.save(m, model_checkpoint_file)
if "label_counts" in m['hyper_parameters']:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'])
else:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)

In [28]:
genes = load_genes(model.organisms)
set(model.genes) - set(genes.index)

/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


set()

In [23]:
#Get indices of genes that exist in genes. index
valid_indices = [i for i in range(len(model.genes)) if model.genes[i] in genes.index]
# Create-new •embedding-weights tensor with only valid genes new _weights -=-model.gene_encoder[01.embeddings.weight[valid_indices]
# Create- new-embedding layer with-correct size
new_embeddings = torch.nn.Embedding(len(valid_indices), model.gene_encoder[0].embeddings.embedding_dim, device=model.device)
#Copy the-weights-to-the new-embedding-layer new_embeddings.weight.data.= new_weights
#-Replace the old •embeddings with the new one
model.gene_encoder[0].embeddings = new_embeddings
model.genes = [i for i in model.genes if i in genes.index]

model = model.to(model.device)

In [29]:
adata = sc.read("../../data/temp/gNNpgpo6gATjuxTE7CCp.h5ad")
max_len = 4000
preprocessor = Preprocessor(
    force_preprocess=True,
    skip_validate=True,
    #drop_non_primary=False,
    do_postp=False
)
preprocessor = Preprocessor(do_postp=False)
adata = preprocessor(adata)
adata.layers['true'] = adata.X.copy()
denoise = Denoiser(
    batch_size=40,
    max_len=max_len,
    max_cells=30_000,
    doplot=False,
    num_workers=8,
    predict_depth_mult=5,
    downsample_expr=0.7,
)
metrics, idx, nadata = denoise(model, adata)
metrics

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 51370 renamining
filtered out 0 cells, 51370 renamining
Removed 82 genes not known to the ontology
Removed 0 duplicate genes
Added 670 genes in the ontology but not present in the dataset
validating


/home/ml4ig1/Documents code/scDataLoader/scdataloader/preprocess.py:281: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adata, organism=adata.obs.organism_ontology_term_id[0], need_all=False


starting QC
Seeing 22952 outliers (44.68% of total dataset):
done
AnnData object with n_obs × n_vars = 51370 × 70704
    obs: 'biosample_id', 'donor_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'cell_type__custom', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'is_primary_data', 'age', 'self_reported_ethnicity_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes'
    var: 'feature_is_filtered', 'feature_name', 'feature_refe

/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/denoise.py(138)__call__()
    136 
    137         pdb.set_trace()
--> 138         with torch.no_grad(), torch.autocast(device_type=device, dtype=self.dtype):
    139             for batch in tqdm(dataloader):
    140                 gene_pos, expression, depth = (

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/denoise.py(139)__call__()
    137         pdb.set_trace()
    138         with torch.no_grad(), torch.autocast(device_type=device, dtype=self.dtype):
--> 139             for batch in tqdm(dataloader):
    140                 gene_pos, expression, depth = (
    141                     batch["genes"].to(device),



  0%|          | 0/750 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fcb96baca60>
Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/scprnt2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()Exception ignored in: 
  File "/home/ml4ig1/miniconda3/envs/scprnt2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fcb96baca60>    
if w.is_alive():Traceback (most recent call last):

  File "/home/ml4ig1/miniconda3/envs/scprnt2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
  File "/home/ml4ig1/miniconda3/envs/scprnt2/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
        self._shutdown_workers()assert self._parent_pid == os.getpid(), 'can only test a child process'

  File "/home/ml4ig1/miniconda3/envs/scprnt2/lib

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/denoise.py(141)__call__()
    139             for batch in tqdm(dataloader):
    140                 gene_pos, expression, depth = (
--> 141                     batch["genes"].to(device),
    142                     batch["x"].to(device),
    143                     batch["depth"].to(device),

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/denoise.py(142)__call__()
    140                 gene_pos, expression, depth = (
    141                     batch["genes"].to(device),
--> 142                     batch["x"].to(device),
    143                     batch["depth"].to(device),
    144                 )

tensor([[21605, 21622, 21624,  ..., 43395, 43407, 44002],
        [21605, 21622, 21624,  ..., 43395, 43407, 44002],
        [21605, 21622, 21624,  ..., 43395, 43407, 44002],
        ...,
        [21605, 21622, 21624,  ..., 43395, 43407, 44002],
        [21605, 21622, 21624,  ..., 43395, 43407, 44002],
        [21605, 21622, 21624

100%|██████████| 750/750 [03:08<00:00,  3.98it/s]


AnnData object with n_obs × n_vars = 30000 × 44741
    obs: 'pred_cell_type_ontology_term_id', 'pred_tissue_ontology_term_id', 'pred_disease_ontology_term_id', 'pred_assay_ontology_term_id', 'pred_self_reported_ethnicity_ontology_term_id', 'pred_sex_ontology_term_id', 'pred_organism_ontology_term_id', 'conv_pred_cell_type_ontology_term_id', 'conv_pred_tissue_ontology_term_id', 'conv_pred_disease_ontology_term_id', 'conv_pred_assay_ontology_term_id', 'conv_pred_self_reported_ethnicity_ontology_term_id'
    obsm: 'scprint_emb'
    layers: 'scprint_mu', 'scprint_theta', 'scprint_pi'


{'reco2noisy': 0.03821803058006762,
 'reco2full': 0.062068988001814755,
 'noisy2full': 0.37211679148539006}

In [ ]:
stdata = stdata[:, ["BLANK" not in i and "NegControl" not in i for i in stdata.var.index.tolist()]]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    stdata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    stdata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)

In [ ]:
# spec xenium 
stdata = stdata[:, ["BLANK" not in i and "NegControl" not in i for i in stdata.var.index.tolist()]]

# set to 300, 1 for visium
sc.pp.filter_cells(stdata, min_counts=50, )
sc.pp.filter_genes(stdata, min_cells=5)

stdata.layers["counts"] = stdata.X.copy()
sc.pp.normalize_total(stdata, inplace=True)
sc.pp.log1p(stdata)
sc.pp.pca(stdata)
sc.pp.neighbors(stdata)
sc.tl.umap(stdata)
sc.tl.leiden(stdata)

In [ ]:
sq.pl.spatial_scatter(
    stdata,
    library_id="spatial",
    shape=None,
    color=[
        "leiden",
        "total_counts",
    ],
    wspace=0.4,
    ncols=1,
)

In [ ]:
stdata = stdata[stdata.obs.leiden != "0", :]
stdata = stdata.copy()
stdata

In [ ]:
# preprocess

In [ ]:
embed = Embedder(how="most var", max_len=500, add_zero_genes=0, num_workers=16, pred_embedding = ["cell_type_ontology_term_id"], keep_all_cls_pred=False, output_expression="none", batch_size=64)
